In [ ]:
import os, sys
sys.path.insert(1, os.getcwd()) 
sys.path.append("..")

In [ ]:
from booklet.tem_1_0_0.signature import  *
from booklet.tem_1_0_0.converters import  *
from booklet.tem_1_0_0.templates import  *

In [8]:
def test(hello=None, *args, **kwargs):
    print(hello)
    print(args, len(args))
    print(kwargs, len(kwargs))

In [9]:
test((1, 2, 3))

(1, 2, 3)
() 0
{} 0


In [10]:
test(1, 2, 3)

1
(2, 3) 2
{} 0


In [11]:
test(r=1, g= 2, b =3)

None
() 0
{'r': 1, 'g': 2, 'b': 3} 3


## Color Test

In [1]:
from reportlab.lib.colors import CMYKColor

In [2]:
black = CMYKColor(1,1,1,0)

In [5]:
black.hexval()

'0x000000'

In [6]:
black.rgb()

(0.0, 0.0, 0.0)

In [7]:
black.cmyk()

(1, 1, 1, 0)

# Signature basic test

In [ ]:
sig_4 = Signature("test.pdf", ".", page_range=None)
print("page_range:", sig_4.page_range)
print("ouput_path:", sig_4.output_path)
print("ouput_file_name:", sig_4.basic_output_file_name)
print("file_name:", sig_4.file_name)
print("modifier infos:", sig_4.modifier_infos)

In [ ]:
print(os.listdir(sig_4.output_path))
sig_4.save_to_file()
print(os.listdir(sig_4.output_path))

# SigComposition Test

In [ ]:
from pprint import pprint

In [ ]:
sig = SigComposition(128, 16)
print(f"{sig.leaves}: {sig.composition}")
print("Layout: ", sig.layout)
pprint(sig.imposition[0])

In [ ]:
pprint(sig.imposition[0])
pprint(sig.imposition[1])

# Converters and Templates

In [28]:
import os, sys
sys.path.insert(1, os.getcwd()) 
sys.path.append("..")
from booklet.tem_1_0_0.signature import  Manuscript
from booklet.tem_1_0_0.converters import  Signature, ToImage, SigComposition
from booklet.tem_1_0_0.templates import  Imposition, PrintingMark, Note

ModuleNotFoundError: No module named 'booklet.tem_1_0_0'

In [6]:
manuscript = Manuscript("test.pdf", ".", filename = None,page_range=None)
sig_composition = SigComposition(4,1)
#manuscript.modifier_register(ToImage(dpi=600))
manuscript.modifier_register(Signature(sig_composition, blank_mode="both", riffle = True, fold=True))
#manuscript.modifier_register(Note())
manuscript.modifier_register(Imposition(True, 30, True, (0.5,0.5,0.5,0.), 20, sig_composition))
manuscript.modifier_register(PrintingMark(100,True, True, True))


In [7]:
manuscript.modifier_infos

[{'Signature': {'type': 'converter',
   'description': 'basic routine for signature generation'}},
 {'Imposition': {'type': 'template', 'description': 'Imposition work'}},
 {'PrintingMark': {'type': 'template',
   'description': 'Add printing marks to manuscript'}}]

In [8]:
manuscript.update(file_mode="safe")

False 2x10 [245.0, 15.0] [245.0, 557.69685] 35.0


'all'

In [9]:
manuscript.save_to_file()

0

In [10]:
printing_mark = manuscript.modifiers[2]

In [16]:
pa = 0.25 * printing_mark.margin
(printing_mark.margin + printing_mark.reg_l+pa , pa )

(512.5, 62.5)

In [6]:
for meta in manuscript.meta:
    value = manuscript.meta[meta]
    print(meta, manuscript.meta[meta], type(value))

/Creator Draw <class 'str'>
/Producer HornPenguin Booklet <class 'str'>
/CreationDate D:20220617175511+09'00' <class 'str'>
/signature 16 <class 'int'>
/Imposition 2x4 <class 'str'>
/ModDate D:20220913223211+09'00' <class 'str'>


example

D:20220617175511+09'00'

D:{2022}{06}{17}{17}{55}{11}+09'00'

D:{year}{month}{date}{time}{hour}{minute}{second}

## Tempfile Test

In [34]:
import os, sys
sys.path.insert(1, os.getcwd()) 
sys.path.append("..")
import io, tempfile
import PyPDF2
from reportlab.pdfgen.canvas import Canvas

In [66]:
temp_pdf_named = tempfile.NamedTemporaryFile(
                mode= "wb",
                prefix=f"{0}_{__name__}_",  
                suffix=".pdf", 
                dir=tem_dir.name,
                )

In [70]:
temp_pdf_named.delete

True

In [ ]:
class NamedTempFile:
    def __init__(self, tempfile:TempFile, mode:Union[None,str], delete:bool = False):
        if tempfile.delete:
            raise ValueError("Named Temporary File must be in non-delete mode.")
        self.path = Path(tempfile.name)
        self.mode = tempfile.mode if mode ==None else mode
        self.delete = delete if type(delete, bool) else False
        tempfile.close()
        os.unlink(self.path)
        self.stream = open(self.path, tempfile.mode)
    def __del__(self, *args):
        self.close()
        if self.delete:
            os.remove(self.path)
    def __exit__(self, *args):
        self.close()
        if self.delete:
            os.remove(self.path)
    @classmethod
    def from_temp_setting(cls, *args, **kwargs):
        kwargs["dir"] = False
        return cls(tempfile.NamedTemporaryFile(args, kwargs))
    @property
    def closed(self):
        return self.stream.closed
    def fileno(self):
        return self.stream.fileno()
    def truncate(self):
        return self.stream.truncate()
    def tell(self):
        return self.stream.tell()
    def isatty(self):
        return self.stream.isatty()
    def flush(self):
        return self.stream.flush()
    def seek(self, *args, **kwargs):
        return self.stream.seek(args, kwargs)
    def close(self):
        return self.stream.close()
    def readable(self):
        return self.stream.readable()
    def read(self, *args, **kwargs):
        return self.stream.read(args, kwargs)
    def readline(self, *args, **kwargs):
        return self.stream.readline(args, kwargs)
    def readlines(self, *args, **kwargs):
        return self.stream.readlines(args, kwargs)
    def writeable(self):
        return self.stream.writable()
    def write(self, *args, **kwargs):
        return self.stream.write(args, kwargs)
    def writeline(self, *args, **kwargs):
        return self.stream.writeline(args, kwargs)
    def writelines(self, *args, **kwargs):
        return self.stream.writelines(args, kwargs)

In [67]:
with open(temp_pdf_named.name, "r") as f:
    st =f.read()

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\HYUNSE~1\\AppData\\Local\\Temp\\tmpqdix4v0p\\0___main___4t8k6j19.pdf'

In [103]:
from pathlib import Path
class NamedTempFile:
    def __init__(self, tempfile, mode=None, delete:bool = False):
        if tempfile.delete:
            raise ValueError("Named Temporary File must be in non-delete mode.")
        self.path = Path(tempfile.name)
        self.mode = tempfile.mode if mode ==None else mode
        self.delete = delete if type(delete) ==bool else False
        tempfile.close()
        #os.unlink(self.path)
        self.stream = open(self.path, self.mode)
    def __del__(self, *args):
        self.close()
        if self.delete:
            os.remove(self.path)
    def __exit__(self, *args):
        self.close()
        if self.delete:
            os.remove(self.path)
    @classmethod
    def from_temp_setting(cls, *args, **kwargs):
        kwargs["delete"] = False
        return cls(tempfile.NamedTemporaryFile(*args, **kwargs))
    @property
    def closed(self):
        return self.stream.closed
    def fileno(self):
        return self.stream.fileno()
    def truncate(self):
        return self.stream.truncate()
    def tell(self):
        return self.stream.tell()
    def isatty(self):
        return self.stream.isatty()
    def flush(self):
        return self.stream.flush()
    def seek(self, *args, **kwargs):
        return self.stream.seek(*args, **kwargs)
    def close(self):
        return self.stream.close()
    def readable(self):
        return self.stream.readable()
    def read(self, *args, **kwargs):
        return self.stream.read(*args, **kwargs)
    def readline(self, *args, **kwargs):
        return self.stream.readline(*args, **kwargs)
    def readlines(self, *args, **kwargs):
        return self.stream.readlines(*args, **kwargs)
    def writeable(self):
        return self.stream.writable()
    def write(self, *args, **kwargs):
        return self.stream.write(*args, **kwargs)
    def writeline(self, *args, **kwargs):
        return self.stream.writeline(*args, **kwargs)
    def writelines(self, *args, **kwargs):
        return self.stream.writelines(*args, **kwargs)

In [113]:
from tempfile import NamedTemporaryFile

tem_dir = tempfile.TemporaryDirectory()
byte_pdf = io.BytesIO()
test_pdf = NamedTempFile.from_temp_setting(
        mode= "rb+",
        prefix=f"{0}_{__name__}_", 
        suffix=".pdf", 
        dir=tem_dir.name,
        delete = False
)

In [114]:
pdf = PyPDF2.PdfFileReader("./test.pdf")

In [115]:
newpdf = PyPDF2.PdfFileWriter()
newpdf.append_pages_from_reader(pdf)

In [116]:
temp_named_pdf_can = Canvas(test_pdf, pagesize = (100,100))

In [117]:
temp_named_pdf_can.rect(0,0,50,50,fill=1)

In [118]:
temp_named_pdf_can.showPage()

In [119]:
temp_named_pdf_can.save()

In [121]:
new3 =  PyPDF2.PdfFileReader(test_pdf)

In [122]:
temp_pdf_named.fileno()

11

In [123]:
isinstance(byte_pdf, io.BytesIO)

True